<center>
    <h1> 
        Pré-processamento do Conjunto de Dados (Transformação)
    </h1>
</center>
    <div style="text-align: right;"><h3>
        Carlos Eduardo Cassimiro da Silva
    </h3></div>

<center>
    <h4>Neste notebook, iremos criar e aplicar o pipeline do pré-processamento das imagens, compilando todos os métodos estudados e verificados anteriormente. O objetivo do pré-processamento é transformar as imagens em histogramas, também tratando as amostras escuras antes disso. </h4>
    Link do dataset: https://github.com/ricardobnjunior/Brazilian-Identity-Document-Dataset
</center>

<h4>Desafio #1: Classificação de documentos (RG, CNH e CPF) </h4>

 - Contextualização: Inúmeras áreas de diferentes organizações (usualmente como parte de um processo de um backoffice) recepcionam documentos dos seus clientes para formação de kits de documentação. Tais kits são, por exemplo, compartilhados com outros stakeholders das empresas. Conforme pode-se pressupor, um desafio nesse cenário refere-se ao fato de que o cliente pode enviar documentos, porém sem necessariamente indicar a qual tipo se o documento se refere (RG, CNH ou CPF, por exemplo). Dessa forma, ao invés de demandar um trabalho manual para essa leitura e classificação dos documentos, podemos construir um modelo de aprendizado que tenha capacidade de ler um conjunto de documentos (em .jpg, por exemplo) e, subsequentemente, realizar a classificação em três tipos distintos: RG, CNH e CPF.
 - Dataset: Para esse desafio utilizaremos um dataset público de RG, CNH e CPF (incluindo as imagens). Este repositório apresenta o conjunto de dados denominado Brazilian Identity Document Dataset (BID Dataset), o primeiro conjunto de dados público de documentos de identificação brasileiros. <br>

<h4>Roteiro</h4>
Módulos utilizados <br>
1. Funções extras <br>
2. Pipeline da transformação e tratamento das imagens <br>
3. Aplicando o pipeline em todas as imagens <br>
4. Salvando o conjunto de dados transformado

##### Módulos utilizados

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import copy
from sklearn.preprocessing import MinMaxScaler
import os
from sys import getsizeof
import json

# 1. Funções extras

In [2]:
def clahe_func(img):
    r, g, b = cv2.split(img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    red = clahe.apply(r)
    green = clahe.apply(g)
    blue = clahe.apply(b)
    return cv2.merge((red, green, blue))

In [3]:
def increase_brightness_percent(img, percent):
    r, g, b = cv2.split(img)
    return cv2.merge((r*(percent*1.002989), g*(percent*1.005870), b*(percent*1.001140))).astype('uint8')

In [4]:
def media_movel(hist, n):
    medias_suavizada = []
    tam = len(hist)

    for i in range(tam-n):
        soma = 0
        for j in range(n):
            soma += hist[i+j]
        medias_suavizada.append(soma/n)
        
    for i in range(tam-1,tam-n,-1):
        soma = 0
        for j in range(n):
            soma += hist[i-j]
        medias_suavizada.append(soma/n)
    
    return medias_suavizada

# 2. Pipeline da transformação e tratamento das imagens

In [5]:
def prepross(image):
    ksize = (70,70) # Máscara do borramento

    image_hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV) # Transformando no formato HSV
    H, _, V =  cv2.split(image_hsv) # Pegando apenas os valores de Hue e Value

    # Verificando se a imagem é escura a partir do Value do HSV
    if np.mean(V) < 175: # Valor obtido experimentalmente no teste 1.5
        # Em caso positivo, aplicamos as técticas de melhoramento (teste 1.1,1.2 e 1.10)
        image = increase_brightness_percent(image, 1.45)
        image = clahe_func(image)
        
    image = cv2.blur(image, ksize) # Em seguida, aplicamos o borramento (teste 1.10)

    # Após melhorar as amostras escuras, criamos os histogramas (teste 1)
    hist = []
    hist.append(cv2.calcHist(image,[0],None,[256],[0,256]))
    hist.append(cv2.calcHist(image,[1],None,[256],[0,256]))
    hist.append(cv2.calcHist(image,[2],None,[256],[0,256]))

    # E aplicamos a normalização dos mesmos (teste 3)
    for i in range(0,3):
        mm = MinMaxScaler()
        hist[i] = mm.fit_transform(hist[i], hist[i])
        hist[i] = media_movel(hist[i],5)
    
    # O cv2.calcHist gera uma lista de lista, precisamos deixar todos os valores em uma única lista
    for i in range(3):
        temp = []
        for j in range(len(hist[i])):
            temp.append(hist[i][j][0])
        hist[i] = media_movel(temp,5) # Aproveitando para suavizar os "picos" com a média móvel (teste 1.9)
    
    # Também testaremos acrescentar o começo do Hue do HSV no "sinal" (teste 1.8)
    img_hsv =  cv2.split(cv2.cvtColor(image,cv2.COLOR_RGB2HSV))
    H = cv2.calcHist(img_hsv,[0],None,[256],[0,256]) # Computando o histograma
    
    # Ajustando o formato da lista
    temp = []
    for j in range(len(H)):
        temp.append(H[i][0])
    
    mm = MinMaxScaler()
    H = mm.fit_transform(H, H)
    H = media_movel(H,5)
    
    # Opção 1: submeter ao modelo somente os histogramas do RGB
    opt1 = hist[0][4:]+hist[1][4:]+hist[2][4:]
    # Opção 2: submeter ao modelo o histograma do RGB + uma faixa do H do HSV
    opt2 = opt1+H[:100]
    
    return opt1, opt2

# 3. Aplicando o pipeline em todas as imagens 

In [6]:
path = 'imagens_originais/BID_Dataset/' # caminho do diretório
folders = os.listdir(path) # os.listdir lista todos os arquivos ou pastas dentro do referido diretório 
op1 = [] # Iremos testas duas opções de dados de treinamento a primeira com os canais do RGB
op2 = [] # e a segunda com RGB + 100 primeiros valores do Hue do HSV
y8 = [] # Também testaremos diferentes targets da classificação para verificar as diferenças
# Não salvei no pré processamento, mas também testaresmo o treinamento com somente os 100 primeiros valores do HSV

In [7]:
for i in folders:   # Percore cada pasta
    if i == 'desktop.ini' :   # Pula esse arquivo oculto do sistema
        continue
    path_att = path+i+'/'    # Atualiza a string do caminho adicionando o nome da pasta
    images = os.listdir(path_att)     # "Puxa" o nome dos arquivos da pasta
    for j in range(2,len(images),3): # Laço para pegar os documentos. Como queremos só as imagens dos documentos,
        img = cv2.imread(path_att+images[j])                                  # então pulamos de 3 em 3 arquivos
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    # Converte para RGB
        opt1, opt2 = prepross(img)   # Aplica o pré-processamento na imagem
        op1.append(opt1)  # Salva a opção de treinamento 1
        op2.append(opt2)  # Salva a opção de treinamento 2
        y8.append(i) # Salva o y com 8 targets

In [8]:
# Com a lista de 8 targets, podemos criar as listas com os outros targets
y3 = []
y6 = []
y7 = []
for i in y8: # Em um único laço, faremos todas as listas com os diferentes targets
    if (i == 'CNH_Aberta') or (i == 'RG_Aberto'):
        y7.append('GRANDE') # Separa os 6 tipos de documentos dos abertos
    else:
        y7.append(i)
    # Separada em 3 tipos de documento, incluindo os abertos
    if (i=='CNH_Aberta') or (i =='CNH_Frente') or (i=='CNH_Verso'):
        y3.append('CNH')
    elif (i=='RG_Aberto') or (i =='RG_Frente') or (i=='RG_Verso'):
        y3.append('RG')
    else:
        y3.append('CPF')
    # Separa em 6 tipos de documentos
    if (i=='CNH_Aberta'):
        y6.append('CNH_Frente')
    elif (i=='RG_Aberto'):
        y6.append('RG_Frente')
    else:
        y6.append(i)

# 4. Salvando o conjunto de dados transformado

In [11]:
# Optei em salvar os arquivos em um dicinário (e depois JSON) pela comonidade dos dicionários
data = {'opt1': op1, 'op2':op2, 'y3':y3,'y6':y6,'y7':y7,'y8':y8}

with open('imagens_originais/desafio1_data2.json', 'w') as outfile: # Salvando os dados em formato JSON em um arquivo
    json.dump(data, outfile)